<a href="https://colab.research.google.com/github/cnboonhan/colab/blob/main/mixtral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run this if in google colab environment to install dependencies
# TODO: Change to using conda
! pkill ollama
! curl https://ollama.ai/install.sh | sh
! pip install llama-index qdrant_client torch transformers pyTelegramBotAPI

import subprocess
subprocess.Popen(["ollama", "serve"])
! ollama pull mistral

In [2]:
from pathlib import Path
import qdrant_client
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    download_loader,
)
from llama_index.llms import Ollama
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.readers import SimpleDirectoryReader

llm = Ollama(model="mistral")

JSONReader = download_loader("SimpleDirectoryReader")
loader = SimpleDirectoryReader(input_dir=Path('./data'))
documents = loader.load_data()
client = qdrant_client.QdrantClient(
    path="./qdrant_data"
)
vector_store = QdrantVectorStore(client=client, collection_name="data")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm,embed_model="local")
index = VectorStoreIndex.from_documents(documents,service_context=service_context,storage_context=storage_context)
query_engine = index.as_query_engine()

/home/cnboonhan/miniconda3/envs/rag_faq_bot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 684/684 [00:00<00:00, 2.83MB/s]
model.safetensors: 100%|██████████| 133M/133M [00:01<00:00, 106MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 2.68MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 80.8MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 62.2MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 636kB/s]


In [4]:
print(query_engine.query("What does apigw.yaml do?").response)
# print(query_engine.query("What does the library code do?").response)
# print(query_engine.query("What does the library code do?").response)

 The provided context information is a section of an AWS CloudFormation template written in YAML format. It defines multiple AWS::CloudWatch::Alarm resources. Each resource sets up a CloudWatch alarm with specific properties, such as its name, description, metric to be monitored (MaxOffsetLag), namespace, period, threshold, comparison operator, and alarm actions when the threshold is exceeded. The alarm actions in each case are set up to send an email notification using an Amazon Simple Notification Service (SNS) topic whenever the specified Kafka message offset lag threshold is breached. The dimensions of the resources specify the consumer group, cluster name, and topic for each respective CloudWatch alarm.


In [ ]:
import telebot
key = input("Provide Telegram Bot Key")
bot = telebot.TeleBot(key, parse_mode=None)
@bot.message_handler(func=lambda m: True)
def echo_all(message):
  response = query_engine.query(message.text)
  bot.reply_to(message, response)
bot.infinity_polling()